LAB 2

Imports

In [30]:
import numpy as np
from dataclasses import dataclass, field
import heapq
import time
import random

In [31]:
np.random.seed(1)
random.seed(1)

Parameters:

In [32]:
#problem to resolve
file_name = "data/problem_g_1000.npy"
array = np.load(file_name)

# number of cities to explore
INDIVIDUAL_VECTOR_SIZE =  array.shape[1]

#data class of the solution, we have the value of the solution and the vector to represent it
# vector: ordered list of cities to visit, vector[i] = 5 = array[5] is the i-th city to visit
@dataclass
class Individual:
    value: float
    vector: list[int]= field(default_factory=lambda: []*INDIVIDUAL_VECTOR_SIZE)
    
    def __lt__(self, other):
        return self.value < other.value

    def __repr__(self):
        return f"Individual({self.vector}, value={self.value})"
    
max_populations_size = 200
generations = 50000
first_population_size = 150
parent_selection_percentage = 0.3 #=30%
mutation_probability = 0.6 #=60%

if "10" in file_name or "20" in file_name or "50" in file_name:
    generations = 1000
if "100" in file_name:
    generations = 10000
if "200" in file_name:
    generations = 40000
if "500" in file_name or "1000" in file_name:
    generations = 50000

Functions:

In [33]:
##THE POPULATIONS IS A HEAP, SO WE ALWAYS INSERT NEGATIVE VALUE TO MAKE THE REMOVE FASTER

#Function to calculate the fitness of an individual
def calculate_fitness(individual: Individual):
    fitness = 0.0
    for i in range(len(individual.vector) - 1):
        city_from = individual.vector[i]
        city_to = individual.vector[i + 1]
        fitness += array[city_from][city_to]
    # Adding the return to the starting city
    fitness += array[individual.vector[-1]][individual.vector[0]]
    individual.value = -fitness

#Inizialization of a random population
def initialize_population(size = first_population_size) -> list[Individual]:
    population = []
    for _ in range(size):
        vector = np.random.permutation(INDIVIDUAL_VECTOR_SIZE).tolist()
        individual = Individual(value=0.0, vector=vector)
        calculate_fitness(individual)
        heapq.heappush(population, individual)
    return population

def parent_selection(population: list[Individual]) -> list[tuple[Individual, Individual]]:
    parents = heapq.nlargest(int(len(population) * parent_selection_percentage), population)
    random.shuffle(parents)
    pairs = [(parents[i], parents[i+1]) for i in range(0, len(parents)-1, 2)]
    return pairs
    
def survivor_selection(population: list[Individual]):
    while len(population) > max_populations_size:
        heapq.heappop(population)
        
def crossover(population: list[Individual], parent1: Individual, parent2: Individual) -> Individual:
    random_index = random.randint(0, INDIVIDUAL_VECTOR_SIZE - 1)
    if random_index == INDIVIDUAL_VECTOR_SIZE - 1:
        pair = (parent2.vector[random_index], parent2.vector[0])
    else:
        pair = (parent2.vector[random_index], parent2.vector[random_index + 1])
   
    child_vector = []
    for city in parent1.vector:
        if city not in pair:
            child_vector.append(city)
        else:
            if city == pair[0]:
                child_vector.append(pair[0])
                child_vector.append(pair[1])
    
    if random.random() < mutation_probability:
        idx1, idx2 = random.sample(range(INDIVIDUAL_VECTOR_SIZE), 2)
        child_vector[idx1], child_vector[idx2] = child_vector[idx2], child_vector[idx1]
        
    child = Individual(value=0.0, vector=child_vector)
    return child        

    
def recombination(population: list[Individual]):
    parents = parent_selection(population)
    for parent1, parent2 in parents:
        child = crossover(population, parent1, parent2)
        calculate_fitness(child)
        heapq.heappush(population, child)
    survivor_selection(population)
    

Resolving code:

In [34]:
population = initialize_population()
count = 0

start = time.time()
for generation in range(generations):
    recombination(population)
    best_individual = heapq.nlargest(1, population)[0]
    if (generation) % (generations/10) == 0:
        print(f"Generation {generation}: Best Value = {-best_individual.value}, Route = {best_individual.vector}")
    if (generation) % (generations/20) == 0:
        mutation_probability *= 0.95
    count += 1
end = time.time()
print(f"Time {end - start} seconds")

print(f"Generation {generation+1}: Best Value = {-best_individual.value}, Route = {best_individual.vector}")

Generation 0: Best Value = 248415.24312096997, Route = [475, 779, 971, 186, 307, 69, 20, 659, 535, 565, 927, 716, 663, 687, 682, 262, 943, 340, 571, 942, 656, 276, 576, 325, 930, 849, 419, 494, 430, 600, 336, 7, 944, 620, 760, 44, 907, 438, 108, 731, 740, 585, 857, 660, 38, 477, 471, 916, 199, 326, 397, 511, 349, 819, 534, 976, 239, 285, 793, 265, 634, 692, 284, 715, 156, 149, 996, 202, 398, 973, 76, 288, 434, 200, 719, 82, 47, 519, 983, 650, 786, 788, 635, 807, 672, 661, 995, 435, 236, 596, 681, 130, 333, 595, 899, 768, 591, 909, 48, 158, 185, 938, 495, 297, 282, 990, 178, 14, 83, 588, 320, 529, 298, 739, 401, 337, 845, 8, 125, 988, 451, 60, 127, 324, 139, 841, 728, 4, 657, 803, 5, 141, 235, 423, 766, 133, 109, 517, 613, 344, 369, 765, 209, 291, 359, 655, 442, 299, 885, 846, 342, 85, 958, 639, 499, 686, 737, 126, 115, 306, 404, 579, 713, 62, 986, 289, 871, 356, 810, 273, 63, 512, 257, 526, 937, 722, 218, 914, 180, 594, 547, 364, 832, 301, 452, 179, 648, 724, 6, 123, 409, 662, 238, 466

In [37]:
max_populations_size = 200
generations = 10000
first_population_size = 150
parent_selection_percentage = 0.4 #=30%
mutation_probability = 0.8 #=60%

population = initialize_population()
count = 0

start = time.time()
for generation in range(generations):
    recombination(population)
    best_individual = heapq.nlargest(1, population)[0]
    if (generation) % (generations/10) == 0:
        print(f"Generation {generation}: Best Value = {-best_individual.value}, Route = {best_individual.vector}")
    if (generation) % (generations/20) == 0:
        mutation_probability *= 0.95
    count += 1
end = time.time()
print(f"Time {end - start} seconds")

print(f"Generation {generation+1}: Best Value = {-best_individual.value}, Route = {best_individual.vector}")

Generation 0: Best Value = 249395.01216535873, Route = [963, 10, 480, 134, 697, 627, 169, 104, 299, 619, 877, 450, 236, 897, 804, 86, 965, 866, 399, 58, 83, 563, 528, 475, 727, 677, 786, 616, 422, 770, 622, 416, 226, 17, 246, 970, 617, 647, 856, 110, 886, 217, 113, 265, 6, 109, 409, 96, 981, 439, 373, 222, 723, 918, 462, 504, 514, 737, 210, 977, 758, 662, 848, 748, 799, 980, 559, 604, 74, 924, 157, 453, 220, 269, 8, 687, 947, 357, 282, 231, 258, 20, 229, 992, 162, 56, 234, 600, 997, 682, 907, 735, 190, 952, 844, 124, 713, 592, 716, 654, 469, 333, 314, 411, 235, 552, 698, 209, 976, 791, 380, 19, 24, 100, 339, 788, 554, 196, 275, 62, 989, 501, 382, 795, 728, 483, 929, 564, 248, 905, 401, 508, 701, 441, 880, 707, 460, 818, 829, 721, 89, 591, 637, 850, 505, 35, 916, 99, 38, 999, 815, 750, 803, 69, 175, 868, 510, 172, 446, 672, 740, 277, 971, 575, 779, 881, 82, 291, 534, 743, 774, 982, 267, 863, 371, 733, 487, 52, 295, 577, 768, 684, 386, 940, 168, 49, 476, 987, 45, 147, 778, 873, 685, 337,

KeyboardInterrupt: 